# Simple simulation

This is a notebook containing a simple simulation
allowing this to be used to assist in the debugging of issues,
or alternatively exploring the abilities of hoomd.

In [1]:
import hoomd
import hoomd.md
from sdrun import molecules

This sets up an initial condition which is very similar to that produced using sdrun,
however this is also simple to see what all the components are and modify them.

In [2]:
context = hoomd.context.initialize('')
trimer = molecules.Trimer()
cell_len = trimer.compute_size()
uc = hoomd.lattice.unitcell(
    N = 1,
    a1 = [cell_len, 0,   0],
    a2 = [0,    cell_len, 0],
    a3 = [0,    0,   1],
    dimensions = 2,
    position = [[0,0,0]],
    type_name = ['R'],
    mass = [1.0],
    moment_inertia = [trimer.moment_inertia]
)
trimer.moment_inertia
system = hoomd.init.create_lattice(uc, 5)
for particle_type in trimer.get_types():
    if particle_type != "R":
        system.particles.types.add(particle_type)

trimer.define_potential()
rigid = trimer.define_rigid()
rigid.create_bodies()

hoomd.md.integrate.mode_standard(dt=0.005)
hoomd.md.integrate.npt(hoomd.group.rigid_center(), kT=0.4, tau=1, P=1, tauP=1)

HOOMD-blue v2.3.0 CUDA (8.0) DOUBLE HPMC_MIXED MPI SSE SSE2 SSE3 
Compiled: 04/25/18
Copyright 2009-2018 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
notice(2): Error calling cudaGetDeviceCount(). No NVIDIA driver is present, or this user
notice(2): does not have readwrite permissions on /dev/nvidia*
notice(2): Ignoring any GPUs in the system.
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 25 particles
notice(2): Notice: slj set d_max=1.0
notice(2): constrain.rigid(): Creating 2

To run the simulation, the `hoomd.run` function needs to be used.

In [3]:
hoomd.run(100)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 100
notice(2): Neighbors included by diameter          : yes
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:00 | Step 100 / 100 | TPS 52029.1 | ETA 00:00:00
Average TPS: 22251.9
---------
-- Neighborlist stats:
1 normal updates / 1 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 4 / n_neigh_avg: 2
shortest rebuild period: 14
-- Cell list stats:
Dimension: 5, 5, 1
n_min    : 4 / n_max: 4 / n_avg: 4
** run complete **


The state of the system can be saved using `system.take_snapshot()`
which then allows the evolution of the system to be tracked and compared.

In [4]:
snapshot = system.take_snapshot()